In [162]:
import pandas as pd
import numpy as np
import re

In [163]:
courses_df = pd.read_csv(filepath_or_buffer="Online_Courses.csv")
data = {"Nombre":courses_df["Title"],"Categoria":courses_df["Category"],"URL":courses_df["URL"],"Descripcion":courses_df["Short Intro"],"Skills":courses_df["Skills"],
        "Instructores":courses_df["Instructors"],"Rating":courses_df["Rating"],"Viewers":courses_df["Number of viewers"]}
df = pd.DataFrame(data=data)
df[["Categoria","Descripcion","Skills","Instructores"]] = df[["Categoria","Descripcion","Skills","Instructores"]].fillna("Other")
df[["Rating","Viewers"]] = df[["Rating","Viewers"]].fillna("0")
df["Rating"] = df["Rating"].map(lambda x: float(x.replace("stars","")))
df["Viewers"] = df["Viewers"].map(lambda x: x.replace(",",""))
df["Viewers"] = df["Viewers"].map(lambda x: re.findall(pattern=r"\d+", string=x)[0])
df["Viewers"] = df["Viewers"].map(lambda x: int(x))

In [164]:
df["Skills"] = df["Skills"].map(lambda x: x.lower())
df["Skills"] = df["Skills"].map(lambda x: x.split(","))

In [165]:
skills = df["Skills"]

skills_copy = []
for i in range(len(skills)):
    lista_aux = []
    for elem in skills[i]:
        elem = re.sub('[^a-zA-Z]+', ' ',elem)
        elem = elem.lstrip()
        lista_aux.append(elem)
    skills_copy.append(lista_aux)

In [166]:
df["Skills"] = skills_copy
skills = df["Skills"]

In [167]:
skills_unique = []
for lista_skills in skills:
    skills_unique += lista_skills
skills_unique = sorted(list(set(skills_unique)))
number_of_skills = len(skills_unique)
# Lista con skills unicos - skills_unique
print(len(skills_unique))

4189


In [168]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
skills_one_hot = ohe.fit_transform(pd.DataFrame(sorted(skills_unique)))

skills_one_hot = skills_one_hot.toarray()

input = skills_one_hot
print(input.shape)
input

(4189, 4189)


array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [169]:
from sklearn.preprocessing import MultiLabelBinarizer

courses_skills = df["Skills"]

mlb = MultiLabelBinarizer()
binarized_data = mlb.fit_transform(courses_skills)
output = binarized_data
print(output.shape)
output = pd.DataFrame(output)
output

(8092, 4189)


,0,1,2,3,4,5,6,7,8,9,...,4179,4180,4181,4182,4183,4184,4185,4186,4187,4188
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8087,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8088,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8089,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8090,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [170]:
len(output)

8092

In [171]:
def user_recomendation_parser(user_skills,output):
    # Creamos una recomendación vacía
    # recomendation = np.zeros(shape=(len(output)))
    # Obtenemos las posiciones donde el usuario tiene una skill
    skill_list = [i for i in range(len(user_skills)) if user_skills[i] == 1]
    
    recomendations = []
    for skill in skill_list:
        
        rows = output.index[output[skill] == 1].tolist()
        recomendations += rows
        
    return recomendations

In [172]:
def parse_recomendations(recomendations,courses_df = None):
    recomendations_info = courses_df.loc[recomendations]
    return recomendations_info.sort_values(by = ["Viewers"],ascending=False)

In [173]:
users = np.load("users.npy")
recomendation = user_recomendation_parser(user_skills=users[0], output=output)
print(recomendation)
len(recomendation)

[3025, 1012, 221, 2076, 3046, 3029, 3046, 3033, 3030, 57, 1071, 2810, 432, 1726, 58, 912, 1064, 323, 3033, 3052, 3002, 3007, 3012, 3013, 3083, 163, 16, 24, 309, 432, 1440, 2260, 3021, 3070, 3024, 3042, 3057, 3059, 172, 3060, 3061, 190, 250, 341, 895, 1969, 2762, 464, 3020, 197, 1498, 365, 2246, 760, 464, 3030, 3031, 403, 2454, 3057, 3059, 3084, 208, 1574, 2262, 2739, 3055, 740, 272, 3053, 32, 284, 411, 676, 714, 743, 807, 833, 1887, 2348, 154, 262, 301, 316, 1332, 1663, 1805, 2452, 911, 1475, 472, 151, 1346, 151, 1346, 123, 696, 1171, 1209, 1683, 2658, 718, 26, 28, 43, 318, 753, 773, 1162, 2036, 2330, 2584, 2644, 3093, 988, 2238, 178, 3005, 3010, 3056, 3087, 3096, 3097, 60, 201, 1773, 2148, 461, 889, 843, 906, 122, 235, 910, 1943, 487, 42, 216, 265, 352, 435, 733, 871, 1194, 1372, 1408, 1416, 1638, 1652, 1695, 2354, 2391, 2428, 2671, 10, 233, 899, 1866, 2098, 3062, 3092, 842, 3092, 453, 429, 72, 1541, 1427, 256, 685, 1056, 3062, 3071, 3076]


174

In [174]:
recomendations = parse_recomendations(recomendations=recomendation,courses_df=df)
recomendations

,Nombre,Categoria,URL,Descripcion,Skills,Instructores,Rating,Viewers
718,Learning How to Learn: Powerful mental tools t...,Personal Development,https://www.coursera.org/learn/learning-how-to...,This course gives you easy access to the inval...,"[test preparation, learning to learn, pomodoro...",Barbara Oakley,4.8,86413
26,Business Analytics Specialization,Data Science,https://www.coursera.org/specializations/busin...,Make Data-Driven Business Decisions. Achieve f...,"[customer analytics, analytics, business analy...","Eric Bradlow ,Peter Fader ,Raghu Iyengar ,Ron ...",4.6,15725
435,Meta Social Media Marketing Professional Certi...,Business,https://www.coursera.org/professional-certific...,Launch your career as a Social Media Marketer....,"[performance advertising, digital marketing, b...",Anke Audenaert,4.9,14105
773,Customer Analytics,Business,https://www.coursera.org/learn/wharton-custome...,Data about our browsing and buying patterns ar...,"[predictive analytics, customer analytics, reg...",Eric Bradlow,4.6,11354
2330,Customer Analytics,Business,https://www.coursera.org/learn/wharton-custome...,Data about our browsing and buying patterns ar...,"[predictive analytics, customer analytics, reg...",Eric Bradlow,4.6,11354
...,...,...,...,...,...,...,...,...
2454,Leading and Developing Top Talent,Business,https://www.coursera.org/learn/leading-and-dev...,This course offers seasoned and up-and-coming ...,[attract and retain a diverse team of employee...,Dr Andrew Campbell,0.0,0
403,Advanced Leadership Skills for the 21st Centur...,Business,https://www.coursera.org/specializations/advan...,Taking your leadership to the next level. Lear...,"[strategic leadership, leading a diverse workf...","Dr Andrew Campbell ,Dr. Loretta Daniels, CDP ,",0.0,0
3031,Growth Product Manager,Other,https://www.udacity.com/course/growth-product-...,Hone specialized Product Management skills in ...,"[activation metrics, growth metrics, customer ...",Other,0.0,0
3030,Become a Natural Language Processing Expert,Other,https://www.udacity.com/course/natural-languag...,Master the skills to get computers to understa...,"[machine translation, word vec text summarizat...",Other,4.5,0
